# Installation de l'environnement

## Installation des dépendances

In [ ]:
!git clone https://github.com/jantic/DeOldify.git DeOldify

Cloning into 'DeOldify'...
remote: Enumerating objects: 2609, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 2609 (delta 87), reused 209 (delta 70), pack-reused 2346
Receiving objects: 100% (2609/2609), 69.71 MiB | 11.11 MiB/s, done.
Resolving deltas: 100% (1170/1170), done.


In [ ]:
cd DeOldify

/content/DeOldify


In [ ]:
!pip install -r requirements-colab.txt
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.4/631.4 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.0/354.0 kB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.8 MB/s eta 0:00:00
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654001 sha256=d8574ac8740be9f44a6750fc0747cfb71dbf36a3b9e12a73952da2685917729b
  Stored in directory: /root/.cac

## Import des modules et modèles

In [ ]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
import os
from PIL import Image
import torch
import fastai
import warnings
import requests
from transformers import BlipProcessor, BlipForConditionalGeneration

In [ ]:
!mkdir ../models

In [ ]:
!wget -O ../models/CBDNet.h5 https://www.dropbox.com/scl/fi/j7jx1avtqwar6jigxlahk/CBDNet.h5?rlkey=i4xh89ru5bqzjvcnllifg6376&dl=0
!wget -O ../models/RIDNet.h5 https://www.dropbox.com/scl/fi/7wgvjocymgfn3yx5xl3sc/RIDNet.h5?rlkey=4d2z97qbunrc42jo4h2lrbsx8&dl=0
!wget -O ../models/InceptionV3.h5 https://www.dropbox.com/scl/fi/3vlglgrongo9b80e3agrw/InceptionV3.h5?rlkey=lwx2cw3xf7gc0x647ts8ie1tr&dl=0
!wget -O ./models/ColorizeStable_gen.pth https://www.dropbox.com/scl/fi/qqpz43g4nm3ovvpkpasuc/ColorizeStable_gen.pth?rlkey=imvmqr28803ox2pimwxx5phnq&dl=0

--2024-01-18 17:46:24--  https://www.dropbox.com/scl/fi/j7jx1avtqwar6jigxlahk/CBDNet.h5?rlkey=i4xh89ru5bqzjvcnllifg6376
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0c7bf9c169661a1f4a673e666c.dl.dropboxusercontent.com/cd/0/inline/CLn0x4iXYnD38XkFSlebuy0nIX1klgQD4hLVrBxvqUHbaCa2ej73qDG_9j8ZyYPqzMaKT9U9udHi8AgZFJB2mv7BxRDjHgoRxomA3QwcjRT3myQAcXdTJH5pWDKXT8AGTtf4X1mkyZtxZGOqFU9fbaWa/file# [following]
--2024-01-18 17:46:25--  https://uc0c7bf9c169661a1f4a673e666c.dl.dropboxusercontent.com/cd/0/inline/CLn0x4iXYnD38XkFSlebuy0nIX1klgQD4hLVrBxvqUHbaCa2ej73qDG_9j8ZyYPqzMaKT9U9udHi8AgZFJB2mv7BxRDjHgoRxomA3QwcjRT3myQAcXdTJH5pWDKXT8AGTtf4X1mkyZtxZGOqFU9fbaWa/file
Resolving uc0c7bf9c169661a1f4a673e666c.dl.dropboxusercontent.com (uc0c7bf9c169661a1f4a673e666c.dl.dropboxusercontent.com)... 162.125.5.15, 2620:100:

In [ ]:
warnings.filterwarnings("ignore",
                        category=UserWarning, message=".*?Your .*? set is empty.*?")

# Pipeline

## Captioning

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

def caption_image(img):
  text = "a photography of"
  inputs = processor(img, text, return_tensors="pt").to("cuda")

  out = model.generate(**inputs)
  return processor.decode(out[0], skip_special_tokens=True)

## Classification

In [ ]:
from deoldify.visualize import *

COLORIZER_MODEL = get_image_colorizer(artistic=False)
CLASSIFIER_MODEL_PATH = "/content/models/InceptionV3.h5"

In [ ]:
categories_list = ["Painting", "Photo", "Schematics", "Sketch", "Text"]

def classify_image(img):
    loaded_model = load_model(CLASSIFIER_MODEL_PATH)
    predictions = loaded_model.predict(img)
    predicted_class_index = np.argmax(predictions)
    confidence = predictions[0][predicted_class_index]

    return categories_list[int(predicted_class_index)]

## Colorisation

In [ ]:
def is_colored_image(img):
    if len(img.shape) == 3 and img.shape[2] == 3:
        if (img[:, :, 0] == img[:, :, 1]).all() and (img[:, :, 0] == img[:, :, 2]).all():
            return False
        return True
    elif len(img.shape) == 2:
        return False
    return False

def colorize(img):
  pil_image = Image.fromarray(np.uint8(img)).convert('RGB')
  input_image_path = 'input_image.jpg'
  pil_image.save(input_image_path)

  image_path = COLORIZER_MODEL.plot_transformed_image(
      path=input_image_path,
      render_factor=35,
      watermarked=False
  )
  colorized_image = Image.open(image_path)
  colorized_image_array = np.array(colorized_image)

  return colorized_image_array

## Débruitage

In [ ]:
def denoise_image(model, noised_image):
    denoised_img_array = model.predict(noised_image)
    denoised_img_array = np.squeeze(denoised_img_array) * 255.0
    denoised_img_array = np.clip(denoised_img_array, 0, 255)
    denoised_img = Image.fromarray(denoised_img_array.astype(np.uint8))

    return denoised_img

def add_noise(image, noise_factor):
    row, col, ch = image.shape
    gauss = np.random.normal(0, 25, (row, col, ch))
    noisy_image = np.clip(image + noise_factor*10 * gauss, 0, 255).astype(np.uint8)
    return noisy_image

### Mesure de la qualité d'une image

In [ ]:
from skimage.metrics import structural_similarity as ssim

def calculate_psnr_color(image1, image2):
    mse = np.mean((image1 - image2) ** 2)
    if mse == 0:  # Les deux images sont identiques
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_snr_color(image1, image2):
    noise = image1 - image2
    signal_power = np.mean(image1 ** 2)
    noise_power = np.mean(noise ** 2)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

def calculate_blur_metric(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray_image, cv2.CV_64F).var()
    return laplacian_var

def is_default_image_better(image_array1, image_array2):
    image1 = np.squeeze(image_array1)
    image1 = (image1 * 255).astype(np.uint8)

    image2 = np.squeeze(image_array2)
    image2 = (image2 * 255).astype(np.uint8)

    # Calculer les métriques pour chaque image
    psnr_value = calculate_psnr_color(image1, image2)
    snr_value = calculate_snr_color(image1, image2)
    ssim_value = ssim(image1, image2, multichannel=True)
    blur_metric_image1 = calculate_blur_metric(image1)
    blur_metric_image2 = calculate_blur_metric(image2)

    # Évaluer quelle image est la meilleure
    score_image1 = 0
    score_image2 = 0

    # PSNR - plus c'est élevé, mieux c'est
    score_image1 += 1 if psnr_value > 26 else 0  # Seuil ajustable

    # SNR - plus c'est élevé, mieux c'est
    score_image1 += 1 if snr_value > 0.5 else 0  # Seuil ajustable

    # SSIM - plus proche de 1, mieux c'est
    score_image1 += 1 if ssim_value > 0.45 else 0  # Seuil ajustable

    # Blur Metric - moins c'est élevé, mieux c'est
    if blur_metric_image1 > blur_metric_image2:
        score_image1 += 1
    else:
        score_image2 += 1

    # Comparer les scores
    if score_image1 > score_image2:
        return True
    else:
        return False

## Preprocess et redimensionnement d'image

In [ ]:
IMG_SIZE = 256 # @param {type:"number"}

model_path_list = ["/content/models/CBDNet.h5","/content/models/RIDNet.h5"]

def preprocess_image(img):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = tf.expand_dims(img_array, 0)

    return img_array

def restore_default_image_size(img, IMG_HEIGHT, IMG_WIDTH):
    img = tf.image.resize(img, (IMG_HEIGHT, IMG_WIDTH), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0

    return img_array

def get_gradio_error(error_message):
    return [
        gr.Label(value=error_message, visible=True),
        gr.Image(visible=False),
        gr.Label(visible=False),
        gr.Image(visible=False),
        gr.Label(visible=False),
        gr.Image(visible=False),
        gr.Label(visible=False),
        gr.Image(visible=False),
        gr.Label(visible=False),
        gr.Label(visible=False)
        ]

# Interface Gradio

In [ ]:
def processing_and_captionning_image(denoise_model_path, target_class, should_colorize_image, noise_factor, img):
    if (img is None or denoise_model_path is None):
      return get_gradio_error("Error: Required field(s) are undefined")

    if (not os.path.exists(denoise_model_path)):
      return get_gradio_error("Error: file " + denoise_model_path + " not found")

    if (not os.path.exists(CLASSIFIER_MODEL_PATH)):
      return get_gradio_error("Error: file " + CLASSIFIER_MODEL_PATH + " not found")

    GRADIO_GLOBAL_RETURN = [
        gr.Label(value="Success: All the required fields have been successfully filled", visible=True),
        gr.Image(img, label="Submited image", visible=True),
    ]

    IMG_HEIGHT, IMG_WIDTH, _ = img.shape

    if(not is_colored_image(img) and should_colorize_image):
      colorized_image = colorize(img)
      GRADIO_GLOBAL_RETURN.extend([
        gr.Label(value="Gray scale image detected (not colored)", visible=True),
        gr.Image(colorized_image, label="Colorized image", visible=True)
      ])
      !rm -f 'input_image.jpg'
    else:
      colorized_image = img
      GRADIO_GLOBAL_RETURN.extend([
        gr.Label(value="Colored image detected, or no need to colorize this image", visible=True),
        gr.Image(visible=False)
      ])

    if (noise_factor > 0):
      noised_image = add_noise(colorized_image, noise_factor)
      GRADIO_GLOBAL_RETURN.append(
        gr.Label(value="Adding noise to the image", visible=True),
      )
    else:
      noised_image = colorized_image
      GRADIO_GLOBAL_RETURN.append(
        gr.Label(value="Image already has noise or should not be noised", visible=True),
      )

    GRADIO_GLOBAL_RETURN.append(
      gr.Image(noised_image, label="Noised image", visible=True)
    )

    processed_img = preprocess_image(noised_image)

    image_class = classify_image(processed_img)
    GRADIO_GLOBAL_RETURN.append(
      gr.Label(value="The image have been detected as: " + image_class, visible=True)
    )

    loaded_denoise_model = load_model(denoise_model_path)
    denoised_image = denoise_image(loaded_denoise_model, processed_img)

    if (IMG_HEIGHT != IMG_WIDTH):
      final_image = restore_default_image_size(denoised_image, IMG_HEIGHT, IMG_WIDTH)
    else:
      final_image = denoised_image

    GRADIO_GLOBAL_RETURN.append(
      gr.Image(final_image, label="De-noised image", visible=True)
    )

    if (image_class == target_class):
      if (is_default_image_better(processed_img, denoised_image)):
        pil_image = Image.fromarray(np.uint8(colorized_image)).convert('RGB')
        GRADIO_GLOBAL_RETURN.append(
          gr.Label(value="Default image is better than de-noised image", visible=True)
        )
      else:
        pil_image = Image.fromarray(np.uint8(final_image)).convert('RGB')
        GRADIO_GLOBAL_RETURN.append(
          gr.Label(value="De-noised image is better than default image", visible=True)
        )
      captioning_predict = caption_image(pil_image)
      GRADIO_GLOBAL_RETURN.append(
        gr.Label(value="The submitted image represents: " + captioning_predict, visible=True)
      )
    else:
      GRADIO_GLOBAL_RETURN.extend([
        gr.Label(visible=False),
        gr.Label(value="As the image have not been classified in the targeted class, there will not be captionning", visible=True)
      ])

    return GRADIO_GLOBAL_RETURN

In [ ]:
ui = gr.Interface(
    fn=processing_and_captionning_image,
    inputs=[
        gr.Dropdown(choices=model_path_list, value=model_path_list[0], label="Select the denoising model"),
        gr.Dropdown(choices=categories_list, value=categories_list[1], label="Select the targeted class"),
        gr.Checkbox(value=True, label="Colorize the image"),
        gr.Slider(minimum=0.0, maximum=0.3, step=0.05, value=0, label="Noise factor"),
        gr.Image(sources=['upload'], label="Supported image types : PNG, JPG, JPEG")
        ],
    outputs=[
        gr.Label(), # Status
        gr.Image(), # Default input image
        gr.Label(), # Is colored ?
        gr.Image(), # If black and white, colored image
        gr.Label(), # Is noise > 0
        gr.Image(), # If noise > 0, noised image
        gr.Label(), # Class of the image
        gr.Image(), # De-noised image
        gr.Label(), # Best image (default or denoised)
        gr.Label() # If image is Photo, captionning details
        ],
    title="Projet Data - Image Captioning",
    description="Upload an image to caption it.",
)

In [ ]:
ui.launch()